# Derive insights on Olympics data using Python Pandas 
### <font color='blue'> Expose an integration point using websockets for orchestration with Node-RED.</font>

## 1. Setup
To prepare your environment, you need to install some packages.

### 1.1 Install the necessary packages

You need the latest versions of these packages:<br>
- websocket-client: is a python client for the Websockets.<br>
- python-swiftclient: is a python client for the Swift API.<br><br>

** Install the websocket client: **

In [21]:
!pip install websocket-client

notebook 5.0.0 requires nbconvert, which is not installed.
ipywidgets 6.0.0 requires widgetsnbextension~=2.0.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


** Install IBM Cloud Object Storage Client: **

In [22]:
!pip install ibm-cos-sdk

notebook 5.0.0 requires nbconvert, which is not installed.
ipywidgets 6.0.0 requires widgetsnbextension~=2.0.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


### 1.2 Import packages and libraries

Import the packages and libraries that you'll use:

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import json, types
import websocket
import _thread
import time
import ibm_boto3
from botocore.client import Config
import codecs
from io import StringIO

## 2. Configuration

Add configurable items of the notebook below

### 2.1 Add your service credentials for Object Storage

You must create Object Storage service on IBM Cloud.
To access data in a file in Object Storage, you need the Object Storage authentication credentials.
Insert the Object Storage authentication credentials as <i><b>credentials_1</b></i> in the following cell after 
removing the current contents in the cell. 

In [24]:
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IBM_API_KEY_ID': 'WVdPkCBKX4EWMLj8F6MAikgYi1HMqAM9MFpiG3bpIndR',
    'IAM_SERVICE_ID': 'iam-ServiceId-9a45c194-d5dd-40ad-8267-fc7607dd6072',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.bluemix.net/oidc/token',
    'BUCKET': 'guraymeetupnotebook-donotdelete-pr-4zwpnmnpnkv1ot',
    'FILE': 'olympics.csv'
}


### 2.3 Global Variables

Add global variables.

In [25]:
olympics_data_filename = 'olympics.csv'
dictionary_data_filename = 'dictionary.csv'

# 3. Persistence and Storage

### 3.1 Configure Object Storage Client

In [26]:
cos = ibm_boto3.client('s3',
                    ibm_api_key_id=credentials_1['IBM_API_KEY_ID'],
                    ibm_service_instance_id=credentials_1['IAM_SERVICE_ID'],
                    ibm_auth_endpoint=credentials_1['IBM_AUTH_ENDPOINT'],
                    config=Config(signature_version='oauth'),
                    endpoint_url=credentials_1['ENDPOINT'])

def get_file(filename):
    '''Retrieve file from Cloud Object Storage'''
    fileobject = cos.get_object(Bucket=credentials_1['BUCKET'], Key=filename)['Body']
    return fileobject

def load_string(fileobject):
    '''Load the file contents into a Python string'''
    text = fileobject.read()
    return text

def put_file(filename, filecontents):
    '''Write file to Cloud Object Storage'''
    resp = cos.put_object(Bucket=credentials_1['BUCKET'], Key=filename, Body=filecontents)
    return resp

def __iter__(self): return 0

# 4. Data 

### 4.1 Prepare data
Combine the olympics and dictionary data into a single dataframe:
- Read olympics data from Object Storage.<br>
- Rename columns<br>
- Populate the data in the dictionary to the Olympics data with a merge<br><br>

In [27]:
body = get_file(olympics_data_filename)
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
olympics = pd.read_csv(body)
olympics = olympics.rename(columns = {'Country':'Code'})
olympics = olympics.rename(columns = {'Year':'Edition'})
body = get_file(dictionary_data_filename)
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
dictionary = pd.read_csv(body)
olympics = pd.merge(olympics, dictionary, on='Code')
olympics.head()

,Edition,City,Sport,Discipline,Athlete,Code,Gender,Event,Medal,Country,Population,GDP per Capita
0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold,Hungary,9844686.0,12363.54346
1,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,1200M Freestyle,Gold,Hungary,9844686.0,12363.54346
2,1896,Athens,Athletics,Athletics,"SZOKOLYI, Alajos",HUN,Men,100M,Bronze,Hungary,9844686.0,12363.54346
3,1896,Athens,Athletics,Athletics,"DANI, Nandor",HUN,Men,800M,Silver,Hungary,9844686.0,12363.54346
4,1896,Athens,Athletics,Athletics,"KELLNER, Gyula",HUN,Men,Marathon,Bronze,Hungary,9844686.0,12363.54346


# 5. Insights on the data using Python Pandas
- Create re-usable functions

In [28]:
def get_medals_gb_year_country():
    """ Group by edition and country and sum medals count.
    """
    medals_groupedBy_yearCountry = olympics.groupby(['Edition','Code']).apply(lambda country: country['Code'].count())
    return medals_groupedBy_yearCountry

def get_medals_gb_year_country_medal():
    """ Group by edition, country, medal type and sum medals count.
    """
    medals_groupedBy_yearCountryMedal = olympics.groupby(['Edition', 'Code', 'Medal']).apply(lambda country: country['Medal'].count())
    return medals_groupedBy_yearCountryMedal

def get_medals_last_10_years(countrycode):
    """ Get Gold, Silver and Bronze medals for a country for last 10 editions.
    """
    last10pics = olympics['Edition'].unique()
    yrs = pd.Series(last10pics).nlargest(10)
    df = pd.DataFrame([], columns=['Year', 'Gold', 'Silver', 'Bronze'])
    medalsdf = get_medals_gb_year_country_medal()
   
    for yr in yrs:
        medaltally = medalsdf[yr][countrycode]
        gold = 0
        silver = 0
        bronze = 0
        if 'Gold' in medaltally:
            gold = medaltally['Gold']
        if 'Silver' in medaltally:
            silver = medaltally['Silver']
        if 'Bronze' in medaltally:
            bronze =  medaltally['Bronze']
        df1 = pd.DataFrame([[yr,gold, silver, bronze]], columns=['Year', 'Gold', 'Silver', 'Bronze'])
        df = df.append(df1, ignore_index=True) 
    df = df.sort_values(by=['Year'], ascending=True)    
    df = df.reset_index()
    del df['index']
    return df

def get_correlation_medalstally():
    """ Get correlation between the medals tally and population, GDP per capita.
    """
    df = get_medals_gb_year_country()
    values  = get_all_olympic_years().values
    size = values.size
    correlations = []
    for i in range(size):
        year = values[i][0]
        df1 = df[year].to_frame(name="Tally")
        df1 = df1.reset_index()
        df2 = pd.merge(df1,dictionary, on='Code')
        corrpop = df2.corr().values[0][1]
        corrgdp = df2.corr().values[0][2]
        resp = {"Year": int(year), "Population":corrpop, "GDP":corrgdp}
        correlations.append(resp)
    return correlations  

def get_medals_category(countrycode, year):
    """ Get the medals count in different sports category for a country in an edition.
    """
    df = olympics[olympics['Edition'] ==  year]
    df1 = df[df['Code'] == countrycode]
    df2 = df1.groupby(['Sport']).apply(lambda country: country['Medal'].count())
    return df2

def get_medals_category_all(countrycode):  
    """ Get the medals count in different sports category for a country for last ten editions.
    """
    df1 = olympics[olympics['Code'] == countrycode]
    df2 = df1.groupby(['Sport']).apply(lambda country: country['Medal'].count())
    return df2

def get_top_ten_gold_tally(year):
    """ Get the top ten gold medal winning countries in an edition.
    """
    df = olympics[olympics['Edition'] ==  year]
    df1 = df[df['Medal'] == 'Gold']
    df2 = df1.groupby(['Code']).apply(lambda country: country['Medal'].count())
    return df2

def get_top_ten_total_tally(year):
    """ Get the top ten total medal winning countries in an edition.
    """
    df = olympics[olympics['Edition'] ==  year]
    df1 = df.groupby(['Code']).apply(lambda country: country['Medal'].count())
    return df1

def get_year_venue():
    """ Get edition venue matrix.
    """
    df = olympics[['Edition', 'City']]
    
    df.head()
    df = df.drop_duplicates()
    df = df.reset_index()
    df = df.set_index('Edition')
    del df['index']
    return df.sort_index()

def get_all_olympic_years():
    """ Get list of all olympic editions.
    """
    df = olympics['Edition']
    df = df.drop_duplicates()
    df = df.reset_index()
    del df['index']
    return df.sort_index()

def get_all_countries():
    """ Get list of all countries.
    """
    df = olympics[['Code','Country']]
    df = df.drop_duplicates()
    df = df.reset_index()
    del df['index']
    return df.sort(['Country'],ascending=[True])

def get_country_edition_data(countrycode,edition):
    """ Get data for a country and edition.
    """
    df = olympics[olympics["Code"] == countrycode]
    df1 = df[df["Edition"] == edition]
    return df1

# 6. Expose integration point with a websocket client 

In [29]:
def on_message(ws, message):
    print(message)
    msg = json.loads(message)
    cmd = msg['cmd']
    
    if cmd == 'MBY':
        country = msg['country']
        tally = get_medals_last_10_years(country)    
        tallyarray=[]
        for i, row in tally.iterrows():
            medaltally = {"Year":int(row["Year"]),
                          "Gold":int(row["Gold"]),
                          "Silver":int(row["Silver"]),
                          "Bronze":int(row["Bronze"])}
            tallyarray.append(medaltally)
        wsresponse = {}
        wsresponse["forcmd"] = "MBY" 
        wsresponse["response"] = tallyarray
        ws.send(json.dumps(wsresponse))
    elif cmd == 'MBSC':
        country = msg['country']
        year = 2008
        response = get_medals_category(country, year)
        
        ct = response.count()
        if ct > 5:
            response = response.nlargest(5)    
        
        medals = []
        categories = []
        for i, row in response.iteritems():
            categories.append(i)
            medals.append(row)   
  
        wsresponse = {}
        wsresponse["forcmd"] = "MBSC"
        wsresponse["response"] = { "categories":categories, "medals":medals}         
        ws.send(json.dumps(wsresponse))
    elif cmd == 'MBSA':
        country = msg['country']
        response = get_medals_category_all(country)
        
        ct = response.count()
        if ct > 5:
            response = response.nlargest(5)    
        
        medals = []
        categories = []
        for i, row in response.iteritems():
            categories.append(i)
            medals.append(row)   
  
        wsresponse = {}
        wsresponse["forcmd"] = "MBSA"
        wsresponse["response"] = { "categories":categories, "medals":medals}         
        ws.send(json.dumps(wsresponse))    
    elif cmd == 'T10G':
        edition = msg["edition"]
        response = get_top_ten_gold_tally(edition)
        ct = response.count()
        if ct > 10:
            response = response.nlargest(10)
        medals = []
        for i, row in response.iteritems():
            data = {"country":i,"tally":row}
            medals.append(data)  
        wsresponse = {}
        wsresponse["forcmd"] = "T10G"
        wsresponse["response"] = medals   
        print(wsresponse)
        ws.send(json.dumps(wsresponse))     
    elif cmd == 'T10M':
        year = msg["edition"]
        response = get_top_ten_total_tally(year)
        ct = response.count()
        if ct > 10:
            response = response.nlargest(10)
        medals = []
        for i, row in response.iteritems():
            data = {"country":i,"tally":row}
            medals.append(data)  
        wsresponse = {}
        wsresponse["forcmd"] = "T10M"
        wsresponse["response"] = medals   
        print(wsresponse)
        ws.send(json.dumps(wsresponse)) 
    elif cmd == 'CORR':
        corr = get_correlation_medalstally() 
        wsresponse = {}
        wsresponse["forcmd"] = "CORR"
        wsresponse["response"] = corr
        ws.send(json.dumps(wsresponse)) 
    elif cmd == 'YV': 
        yearvenue = get_year_venue()
        print(yearvenue.head())
        print('merve')
        yearvenuearray = []
        for i in range(yearvenue.size):
            value = {"Year":int(yearvenue.index[i]),"Venue":yearvenue.values[i].tolist()[0]}
            yearvenuearray.append(value)
        responsejson = {}
        responsejson["forcmd"]="YV"
        responsejson["response"]=yearvenuearray
        ws.send(json.dumps(responsejson))               
    elif cmd == 'DATA':
        country = msg['country']
        edition = msg['edition']
        olympicsslice = get_country_edition_data(country,edition)
        data = []
        numofcolumns = olympicsslice.columns.size
        cols = []
        values = []
        for column in olympicsslice.columns:
            cols.append(column)
        for value in olympicsslice.values:
            values.append(value.tolist()) 
        data = {"cols":cols,"vals":values}    
        responsejson = {}
        responsejson['forcmd']='DATA'
        responsejson['response']= data
        ws.send(json.dumps(responsejson)) 
    elif cmd == 'EDITIONS':
        years = get_all_olympic_years()
        yearsarray = []
        for i,row in years.iteritems():
            for value in row:
                yearsarray.append(value)
        length = len(yearsarray)
        wsresponse = []
        for i in range(length):
            year = {"text":yearsarray[i],"value":yearsarray[i]}
            wsresponse.append(year)
        responsejson = {}
        responsejson['forcmd']='EDITIONS'
        responsejson['response']= wsresponse 
        ws.send(json.dumps(responsejson)) 
    elif cmd == 'COUNTRIES':
        countries = get_all_countries()
        countriesarray = []
        codearray = []
        for i,row in countries.iteritems():
            if i=='Code':
                for value in row:
                    codearray.append(value)
            elif i=='Country':  
                for value in row:
                    countriesarray.append(value)
        length = len(codearray)
        wsresponse = []
        for i in range(length):
            country = {"text":countriesarray[i],"value":codearray[i]}
            wsresponse.append(country)
        responsejson = {}
        responsejson['forcmd']='COUNTRIES'
        responsejson['response']= wsresponse 
        ws.send(json.dumps(responsejson))  

def on_error(ws, error):
    print(error)

def on_close(ws):
    ws.send("DSX Listen End")

def on_open(ws):
    def run(*args):
        for i in range(10000):
            hbeat = '{"cmd":"Olympics DSX HeartBeat"}'
            ws.send(hbeat)
            time.sleep(100)
            
    _thread.start_new_thread(run, ())


def start_websocket_listener():
    websocket.enableTrace(True)
    ws = websocket.WebSocketApp("ws://guray-noderedstarter.eu-gb.mybluemix.net/ws/orchestrate",
                              on_message = on_message,
                              on_error = on_error,
                              on_close = on_close)
    ws.on_open = on_open
    ws.run_forever()

# 7. Start websocket client

In [ ]:
start_websocket_listener()

--- request header ---
GET /ws/orchestrate HTTP/1.1
Upgrade: websocket
Connection: Upgrade
Host: guray-noderedstarter.eu-gb.mybluemix.net
Origin: http://guray-noderedstarter.eu-gb.mybluemix.net
Sec-WebSocket-Key: ataz+dCNm3ViRa/+uiigJA==
Sec-WebSocket-Version: 13


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Connection: Upgrade
Sec-WebSocket-Accept: jy7m35nWFS5o0Bqz2gfXcAAHTQ0=
Date: Mon, 24 Dec 2018 13:23:27 GMT
X-Global-Transaction-ID: 1863416655
Upgrade: websocket
-----------------------
send: b'\x81\xa0\x1b\xd3\x8b\xcd`\xf1\xe8\xa0\x7f\xf1\xb1\xefT\xbf\xf2\xa0k\xba\xe8\xbe;\x97\xd8\x95;\x9b\xee\xaci\xa7\xc9\xa8z\xa7\xa9\xb0'


{"cmd":"Olympics DSX HeartBeat"}


error from callback <function on_message at 0x7f2fe9a41268>: 'DataFrame' object has no attribute 'sort'
  File "/gpfs/fs01/user/s804-ef1df6d2a1e291-2ef1a5e3ed71/.local/lib/python3.5/site-packages/websocket/_app.py", line 345, in _callback
    callback(self, *args)
  File "<ipython-input-29-7b36fb91e4b9>", line 138, in on_message
    countries = get_all_countries()
  File "<ipython-input-28-1a6fc369f879>", line 115, in get_all_countries
    return df.sort(['Country'],ascending=[True])
  File "/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/pandas/core/generic.py", line 3614, in __getattr__
    return object.__getattribute__(self, name)
send: b'\x81\xfe\x03ia\xb5\x0f^\x1a\x97i1\x13\xd6b:C\x8f/|$\xf1F\n(\xfaA\rC\x99/|\x13\xd0|.\x0e\xdb|;C\x8f/\x05\x1a\x97{;\x19\xc1-dA\x847gW\x99/|\x17\xd4c+\x04\x975~P\x8d6h\x1c\x99/%C\xc1j&\x15\x975~P\x8c?nM\x95-(\x00\xd9z;C\x8f/oX\x85?#M\x95t|\x15\xd0w*C\x8f/oX\x85;rA\x97y?\r\xc0j|[\x95>gQ\x81rrA\xce-*\x04\xcd{|[\x9

{"cmd":"Client connected"}
{"cmd":"COUNTRIES"}
{"cmd":"EDITIONS"}
{"cmd":"YV"}
              City
Edition           
1896        Athens
1900         Paris
1904      St Louis
1908        London
1912     Stockholm
merve
{"cmd":"CORR"}


/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/pandas/core/indexes/multi.py:848: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return self._engine.get_value(s, k)
send: b'\x81\xfe\x08j\xd3\xae\xfb\xbe\xa8\x8c\x9d\xd1\xa1\xcd\x96\xda\xf1\x94\xdb\x9c\x90\xe1\xa9\xec\xf1\x82\xdb\x9c\xa1\xcb\x88\xce\xbc\xc0\x88\xdb\xf1\x94\xdb\xe5\xa8\x8c\xa2\xdb\xb2\xdc\xd9\x84\xf3\x9f\xc3\x87\xe5\x82\xdb\x9c\x94\xea\xab\x9c\xe9\x8e\xd6\x8e\xfd\x9a\xce\x8b\xe3\x9d\xca\x8d\xe4\x9f\xca\x8f\xe3\x9b\xcb\x88\xe4\x9d\xd7\x9e\xf1\xfe\x94\xce\xa6\xc2\x9a\xca\xba\xc1\x95\x9c\xe9\x8e\xcb\x90\xe2\x98\xcc\x8c\xe0\x9b\xcb\x8c\xeb\x9e\xce\x8d\xe6\x9e\xc2\x86\xe7\xd3\xd7\x9e\xa8\x8c\xa2\xdb\xb2\xdc\xd9\x84\xf3\x9f\xc2\x8e\xe3\x82\xdb\x9c\x94\xea\xab\x9c\xe9\x8e\xd6\x8e\xfd\x9e\xcb\x8e\xe4\x9e\xc9\x8b\xe3\x98\xc9\x89\xe0\x9d\xcd\x8d\xe7\x99\xc2\x8f\xff\x8e\xd9\xee\xbc\xde\x8e\xd2\xb2\xda\x92\xd1\xbd\x

...

{"cmd":"Olympics DSX HeartBeat"}


send: b'\x81\xa0\xc8\xbe!_\xb3\x9cB2\xac\x9c\x1b}\x87\xd2X2\xb8\xd7B,\xe8\xfar\x07\xe8\xf6D>\xba\xcac:\xa9\xca\x03"'


{"cmd":"Olympics DSX HeartBeat"}


send: b'\x81\xa0\xb82\xcb\xce\xc3\x10\xa8\xa3\xdc\x10\xf1\xec\xf7^\xb2\xa3\xc8[\xa8\xbd\x98v\x98\x96\x98z\xae\xaf\xcaF\x89\xab\xd9F\xe9\xb3'


{"cmd":"Olympics DSX HeartBeat"}
